**Task 07: Querying RDF(s)**

In [10]:
%pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2023-2024/master/Assignment4/course_materials"

Note: you may need to restart the kernel to use updated packages.


First let's read the RDF file

In [60]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")



ns =Namespace("http://somewhere#")

**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [61]:
# TO DO
# Visualize the results

for s,p,o in g.triples((None,RDFS.subClassOf,ns.LivingThing)):
    print(s)


http://somewhere#Person
http://somewhere#Animal


In [73]:
from rdflib.plugins.sparql import prepareQuery


q1 = prepareQuery('''
  SELECT ?subclass WHERE { 
    ?subclass rdfs:subClassOf ns:LivingThing. 
  }
  ''',initNs = { "ns": Namespace("http://somewhere#"),"rdfs": RDFS}
)


for r in g.query(q1):
  print(r.subclass)

http://somewhere#Person
http://somewhere#Animal


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [155]:
# TO DO
# Visualize the results

def list_elems_recursive(elem):
    results =[]
    classes=[elem]
    while classes:
        elem = classes.pop()
        for s,p,o in g.triples((None,RDF.type,elem)):
            results.append((s,p,o))
        for s,p,o in g.triples((None,RDFS.subClassOf,elem)):
            classes.append(s)
    return results



for elem in list_elems_recursive(ns.Person):
    print(elem[0])
    
    
    
for s, p, o in g:
    print(s,p,o)

       

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JimGonzalez
http://somewhere#JaneSmith
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#RockySmith http://xmlns.com/foaf/0.1/knows http://somewhere#SaraJones
http://somewhere#Person http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#LivingThing
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/Given Rocky
http://somewhere#PhDstudent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher

In [154]:
q1 = prepareQuery('''
  SELECT ?individual  WHERE { 
    
    {
      ?individual rdf:type ?subclass.
      optional{?subclass rdfs:subClassOf ?subclass.}
      ?subclass rdfs:subClassOf ns:Person
    }
    UNION
    {
      ?individual rdf:type ns:Person.
    }
    
  }
  ''',initNs = { "ns": Namespace("http://somewhere#"),"rdfs": RDFS,"rdf":RDF}
)


for r in g.query(q1):
  print(r.individual)



http://somewhere#JaneSmith
http://somewhere#JimGonzalez
http://somewhere#SaraJones
http://somewhere#JohnSmith


**TASK 7.3: List all individuals of "Person" or "Animal" and all their properties including their class with RDFLib and SPARQL. You do not need to list the individuals of the subclasses of person**


In [101]:
# TO DO
# Visualize the results

for elem,p,o in g.triples((None,RDF.type,ns.Person)):
    print("Element:",elem)
    for s,p,o in g.triples((elem,None,None)):
        print("Propertie:",p,o)
for elem,p,o in g.triples((None,RDF.type,ns.Animal)):
    print("Element:",elem)
    for s,p,o in g.triples((elem,None,None)):
        print("Propertie:",p,o)

Element: http://somewhere#SaraJones
Propertie: http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
Propertie: http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith
Propertie: http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
Element: http://somewhere#JohnSmith
Propertie: http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/Given John
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
Element: http://somewhere#RockySmith
Propertie: http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Animal
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/Given Rocky
Propertie: http://www.w3.org/2001/vcard-rdf/3.0/FN Rocky Smith
Propertie: htt

In [165]:
q1 = prepareQuery('''
  SELECT distinct ?individual ?property ?value WHERE {   
    {
      ?individual rdf:type ns:Person.
      ?individual ?property ?value
    }
    UNION
    {
      ?individual rdf:type ns:Animal.
      ?individual ?property ?value
    }
      
  }
  ''',initNs = { "ns": Namespace("http://somewhere#"),"rdfs": RDFS,"rdf":RDF}
)


for r in g.query(q1):
  print(r.individual,r.property,r.value)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#RockySmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Animal
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/Give

**TASK 7.4:  List the name of the persons who know Rocky**

In [104]:
# TO DO
# Visualize the results
from rdflib import FOAF
#con el # no funciona si imprimes todo el grafo esta con / no con #
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0/")
rocky =ns.RockySmith
for person,p,o in g.triples((None,FOAF.knows,rocky)):
    print(g.value(subject=person, predicate=VCARD.Given, object=None))
    


Sara
Jane


**Task 7.5: List the entities who know at least two other entities in the graph**

In [113]:
# TO DO
# Visualize the results
for person,p,o in g.triples((None,FOAF.knows,None)):
  list=[]
  for elem in g.triples((person,FOAF.knows,None)):
    list.append(elem[2])
  if len(list)>=2 :
    for elem in list:
      print(person,"knows",elem)
  print("")



http://somewhere#RockySmith knows http://somewhere#JaneSmith
http://somewhere#RockySmith knows http://somewhere#SaraJones

http://somewhere#SaraJones knows http://somewhere#RockySmith
http://somewhere#SaraJones knows http://somewhere#JaneSmith

http://somewhere#RockySmith knows http://somewhere#JaneSmith
http://somewhere#RockySmith knows http://somewhere#SaraJones

http://somewhere#JaneSmith knows http://somewhere#RockySmith
http://somewhere#JaneSmith knows http://somewhere#SaraJones

http://somewhere#SaraJones knows http://somewhere#RockySmith
http://somewhere#SaraJones knows http://somewhere#JaneSmith

http://somewhere#JaneSmith knows http://somewhere#RockySmith
http://somewhere#JaneSmith knows http://somewhere#SaraJones

